# Siddon's Algorithm for Ray Tracing 
Here I will be attempting to implement my own version of Siddon's Algorithm. Very much an attempt.

### THE ALGORITHM
- Calculate range of parametric values $(\alpha_{min},\alpha_{max})$
- Calculate range of indices $(i_{min},i_{max}),(j_{min},j_{max}),(k_{min},k_{max})$
- Calculate parametric sets $\{\alpha_x\},\{\alpha_y\},\{\alpha_z\}$
- Merge sets to form set $\{\alpha\}$
- Calculate voxel lengths
- Calculate voxel indices

In [24]:
%pylab ipympl

Populating the interactive namespace from numpy and matplotlib


#### Calculate Range of Parametric Values

In [141]:
def alpha_func(plane,coor1,coor2):
    '''
    plane is assumed to be already calculated
    '''
    return (plane-coor1)/(coor2-coor1)

In [142]:
def plane_func(index,plane1,d):
    '''
    Parameters:
    ----------
    index :: integer
      index to evaluate at, index in (1,...,n)
    
    plane1 :: float 
      location of plane with index 1 
    
    d :: float 
      distance between planes
    
    Returns:
    -------
    plane_location :: float 
      the location of the plane of the specified index
    
    '''
    return plane1 + (index-1)*d

In [149]:
'''
Parameters
'''
# for a CT array of (Nx-1,Ny-1,Nz-1) voxels
Nx = 5 

Ny = 4
Nz = 4

# distances between the x,y,z planes (also the lengths of the sides of the voxels)
dx = 0.1
dy = 2
dz = 0.5

# initial and final coordinates of the beam
x1,x2 = (0.5,0)
y1,y2 = (3,0)
z1,z2 = (2,0.25)

# initial plane coordinates
xplane1 = 0
yplane1 = 0
zplane1 = 0

'''
Setting up Dictionary 
'''
# MAKE A DESCRIPTION OF EACH OF THE QUANTITIES OF THE DICTIONARY

coor_values = {'x':{},'y':{},'z':{}}

coor_values['x']['N'] = Nx
coor_values['y']['N'] = Ny
coor_values['z']['N'] = Nz

coor_values['x']['d'] = dx
coor_values['y']['d'] = dy
coor_values['z']['d'] = dz

coor_values['x']['1,2'] = (x1,x2)
coor_values['y']['1,2'] = (y1,y2)
coor_values['z']['1,2'] = (z1,z2)

coor_values['x']['plane'] = [xplane1] # this ends up being min,max
coor_values['y']['plane'] = [yplane1]
coor_values['z']['plane'] = [zplane1]

for key in coor_values.keys():
    coor_values[key]['plane'].append(plane_func(coor_values[key]['N'],coor_values[key]['plane'][0],coor_values[key]['d']))

for key in coor_values.keys():
    if coor_values[key]['1,2'][1] - coor_values[key]['1,2'][0] != 0:
        coor_values[key]['alpha_minmax'] = (alpha_func(coor_values[key]['plane'][0],coor_values[key]['1,2'][0],coor_values[key]['1,2'][1]),alpha_func(coor_values[key]['plane'][-1],coor_values[key]['1,2'][0],coor_values[key]['1,2'][1]))
    else:
        coor_values[key]['alpha_minmax'] = (0,1) # set to this so that it doesn't affect later business

alpha_min = max(0,min(coor_values['x']['alpha_minmax']),min(coor_values['y']['alpha_minmax']),min(coor_values['z']['alpha_minmax']))
alpha_max = min(1,max(coor_values['x']['alpha_minmax']),max(coor_values['y']['alpha_minmax']),max(coor_values['z']['alpha_minmax']))

print(alpha_min,alpha_max)


0.2857142857142857 1


#### Calculate Range of Indices

In [144]:
# FOR SOME REASON IT WORKS IF I ROUND THE INDICES BUT I DON'T LIKE THAT 

for key in coor_values.keys():
    if coor_values[key]['1,2'][1] - coor_values[key]['1,2'][0] >= 0:
        indmin = coor_values[key]['N'] - (coor_values[key]['plane'][-1]-alpha_min*(coor_values[key]['1,2'][1] - coor_values[key]['1,2'][0])-coor_values[key]['1,2'][0])/coor_values[key]['d']
        indmax = 1 - (coor_values[key]['plane'][0]-alpha_max*(coor_values[key]['1,2'][1] - coor_values[key]['1,2'][0])-coor_values[key]['1,2'][0])/coor_values[key]['d']
        indmin = int(ceil(indmin))
        indmax = int(floor(indmax))
        coor_values[key]['indminmax'] = (indmin,indmax)
    else:
        indmin = coor_values[key]['N'] - (coor_values[key]['plane'][-1]-alpha_max*(coor_values[key]['1,2'][1] - coor_values[key]['1,2'][0])-coor_values[key]['1,2'][0])/coor_values[key]['d']
        indmax = 1 - (coor_values[key]['plane'][0]-alpha_min*(coor_values[key]['1,2'][1] - coor_values[key]['1,2'][0])-coor_values[key]['1,2'][0])/coor_values[key]['d']
        indmin = int(ceil(indmin))
        indmax = int(floor(indmax))
        coor_values[key]['indminmax'] = (indmin,indmax)

print(coor_values['x']['indminmax'])
print(coor_values['y']['indminmax'])
print(coor_values['z']['indminmax'])


(1, 4)
(1, 2)
(2, 4)


#### Calculate Parametric Sets

In [145]:
alpha_coor_set = {}

for key in coor_values.keys():
    if coor_values[key]['1,2'][1] - coor_values[key]['1,2'][0] > 0:
        print(array([n for n in range(coor_values[key]['indminmax'][0],coor_values[key]['indminmax'][1]+1)]))
        coor_values[key]['alpha_set'] = alpha_func(plane_func(array([n for n in range(coor_values[key]['indminmax'][0],coor_values[key]['indminmax'][1]+1)]),coor_values[key]['plane'][0],coor_values[key]['d']),coor_values[key]['1,2'][0],coor_values[key]['1,2'][1])
    elif coor_values[key]['1,2'][1] - coor_values[key]['1,2'][0] < 0:
        print(array([n for n in range(coor_values[key]['indminmax'][1],coor_values[key]['indminmax'][0]-1,-1)]))
        coor_values[key]['alpha_set'] = alpha_func(plane_func(array([n for n in range(coor_values[key]['indminmax'][1],coor_values[key]['indminmax'][0]-1,-1)]),coor_values[key]['plane'][0],coor_values[key]['d']),coor_values[key]['1,2'][0],coor_values[key]['1,2'][1])
    else:
        coor_values[key]['alpha_set'] = []
    print(coor_values[key]['alpha_set'])


[4 3 2 1]
[0.4 0.6 0.8 1. ]
[2 1]
[0.33333333 1.        ]
[4 3 2]
[0.28571429 0.57142857 0.85714286]


#### Merge Sets to Form Alpha Set

In [146]:
# I still don't think its right bc alpha_min = 2/7 and there is a 0.2 in here which is smaller 
# also there are nfinal+1 values ???
# FIGURE THIS OUT FIRST 
alpha = {alpha_min,alpha_max}.union(set(coor_values['x']['alpha_set'])).union(set(coor_values['y']['alpha_set'])).union(set(coor_values['z']['alpha_set']))
alpha = sort(list(alpha))
alpha


array([0.28571429, 0.33333333, 0.4       , 0.57142857, 0.6       ,
       0.8       , 0.85714286, 1.        ])

#### Calculate Voxel Lengths

In [148]:
nfinal = 1
d12 = 0
for key in coor_values.keys():
    nfinal += coor_values[key]['indminmax'][1] - coor_values[key]['indminmax'][0] + 1 
    d12 += (coor_values[key]['1,2'][1] - coor_values[key]['1,2'][0])**2
d12 = sqrt(d12)

d12,nfinal

(3.5089172119045497, 10)

In [51]:
def voxel_length(alpha,index,d12):
    '''
    Parameters:
    ----------
    alpha :: list
      list of alpha values
    
    index :: integer
      index to evaluate at, index in (1,...,n)
    
    d12 :: float
      distance from point one to point two
    
    Returns:
    -------
    voxel_length :: float
      voxel intersection length
      
    '''
    
    return d12*(alpha[index]-alpha[index-1])

#### Calculate Voxel Indices

In [38]:
def voxel_index(plane,coor1,coor2,alpha_mid,distance):
    '''
    Parameters:
    ----------
    plane :: float????
      coordinate plane (1)
    
    coor1 :: float
      coordinate one
    
    coor2 :: float
      coordinate two
    
    alpha_mid :: float
      average of alpha[m] and alpha[m-1]
    
    distance :: float
      distance between two planes
    
    Returns:
    -------
    voxel_index :: integer
      voxel index for the specific coordinate 
    
    '''
    
    return 1 + (coor1 + alpha_mid(coor2-coor1)-plane)/distance

## Sum it all up 
Here I must diverge from the paper; this is where the superposition/convolution algorithm goes.